# SNe Ia Simulations

The goal of this notebook is to carry out a set of spectral simulations which include SNe Ia templates in the host-galaxy spectra.

#### Interactive queue:
  salloc -N 1 -p debug -L project
  srun -n 1 -c 64 python bgs-sims.py --sim 4 --nproc 64 --zfind

#### Regular queue:
  [see the bgs-sims-zfind.sh script]
  sbatch bgs-sims-zfind.sh
  squeue -u ioannis
  sqs -u ioannis
  
##### Simulation 1:
  Nominal exposure time.
  Fixed r mag (19.5)
  Fixed moon-zenith (30 deg)
  Fixed moon-phase (0.25)
  Vary moon-angle (0-150)


In [17]:
import os
import numpy as np

import fitsio
import healpy as hp
from astropy.table import Table

In [2]:
import desisim.io
import desispec.io
import desimodel.io 

from desisim.obs import new_exposure
from desisim.scripts import quickgen
from desisim.templates import specify_galparams_dict

In [3]:
#import desimodel.io
#import desimodel.footprint
#from redrock.external.desi import rrdesi
#from desispec.parallel import use_mpi
#import desispec.scripts.group_spectra as group_spectra

In [4]:
%matplotlib inline

### Simulation number and input priors

In [5]:
sim = 1
program = 'BGS'

seed = 678245
nexp = 5
nspec = 10

In [34]:
rmagrange = (19.5, 19.5)
zrange = (0.1, 0.3)
sne_rfluxratiorange = (0.5, 1.0)

In [35]:
airmass_range = (1.0, 1.0)
seeing_range = (1.0, 1.0)
moon_phase_range = (0.25, 0.25)
moon_zenith_range = (30, 30)
exptime_range = (300, 300)
moon_angle_range = (0, 150)

In [8]:
simint = int(sim)
simname = '{:02d}'.format(simint)
night = '2017{}{}'.format(simname, simname)
expids = np.arange(nexp) + simint*1000
print('Simulation name = {}, Night = {}, Number of exposures = {}'.format(simname, night, nexp))
print('Exposure IDs = {}'.format(expids))

Simulation name = 01, Night = 20170101, Number of exposures = 5
Exposure IDs = [1000 1001 1002 1003 1004]


### Define the I/O simulation directories.

In [9]:
os.environ['PIXPROD'] = 'snesim-{}'.format(simname)
os.environ['SPECPROD'] = 'snesim-{}'.format(simname)
os.environ['DESI_SPECTRO_DATA'] = desisim.io.simdir()

print('Simulated raw data will be written to {}'.format(desisim.io.simdir()))
print('Pipeline will read raw data from {}'.format(desispec.io.rawdata_root()))
print('Pipeline will write processed data to {}'.format(desispec.io.specprod_root()))

Simulated raw data will be written to /Users/ioannis/work/desi/spectro/sim/snesim-01/
Pipeline will read raw data from /Users/ioannis/work/desi/spectro/sim/snesim-01/
Pipeline will write processed data to /Users/ioannis/work/desi/spectro/redux/snesim-01


### Assign exposures to different tiles.

This step is necessary to keep the spectra from all stacking on the default tile, and ensures that each tile will have a dedicated set of observing conditions.

In [10]:
tiles = desimodel.io.load_tiles()
xyz = hp.ang2vec(tiles['ra'], tiles['dec'], lonlat=True)
center = hp.ang2vec(130, 18, lonlat=True)
d2 = ((xyz - center)**2).sum(axis=1)
tiles = tiles[np.argsort(d2)[0:nexp]]
print('Selecting the following tiles:')
Table(tiles)

Selecting the following tiles:


TILEID,RA,DEC,PASS,IN_DESI,EBV_MED,AIRMASS,STAR_DENSITY,EXPOSEFAC,PROGRAM,OBSCONDITIONS
int32,float64,float64,int16,int16,float32,float32,float32,float32,str6,int32
22594,130.08,18.03,3,1,0.0255654,1.06033,2664.73,1.25706,DARK,1
40476,129.32,17.9,7,1,0.0281447,1.06097,2753.01,1.27791,BRIGHT,4
16832,128.99,18.05,2,1,0.0298504,1.06023,2795.41,1.29012,DARK,1
28356,131.17,18.02,4,1,0.0231459,1.06038,2514.4,1.23876,GRAY,2
11069,130.26,16.58,1,1,0.0251414,1.06783,2711.23,1.26491,DARK,1


### Generate nexp uniform samplings of the simulation parameters.

In [11]:
rand = np.random.RandomState(seed)

In [12]:
seeds = rand.randint(2**32, size=nexp)
seeings = rand.uniform(seeing_range[0], seeing_range[1], nexp)
exptimes = rand.uniform(exptime_range[0], exptime_range[1], nexp)
airmasses = rand.uniform(airmass_range[0], airmass_range[1], nexp)
moonphases = rand.uniform(moon_phase_range[0], moon_phase_range[1], nexp)
moonangles = rand.uniform(moon_angle_range[0], moon_angle_range[1], nexp)
moonzeniths = rand.uniform(moon_zenith_range[0], moon_zenith_range[1], nexp)

In [42]:
allobj_specs = dict()
allobj_specs[program] = specify_galparams_dict(
    program, 
    zrange = zrange,
    magrange = rmagrange, 
    sne_rfluxratiorange = sne_rfluxratiorange,
    add_SNeIa = True,
)
print(allobj_specs[program])

TypeError: specify_galparams_dict() got an unexpected keyword argument 'add_SNeIa'

In [14]:
for ii in range(1):
    obsconditions = {'SEEING' : seeings[ii], 'EXPTIME': exptimes[ii], 'AIRMASS': airmasses[ii],
                     'MOONFRAC': moonphases[ii], 'MOONALT': moonzeniths[ii], 'MOONSEP': moonangles[ii]}
    
    sim, fibermap, meta, obsconditions = new_exposure(program=program, seed=seeds[ii],
                                                      nspec=nspec, night=night,
                                                      expid=expids[ii], tileid=tiles[ii][0],
                                                      obsconditions=obsconditions, 
                                                      specify_targets=allobj_specs,
                                                      overwrite=True)
    
    quickgen_args = quickgen.parse([
        '--simspec', desisim.io.findfile('simspec', night, expids[ii]),
        '--fibermap', desisim.io.findfile('simfibermap', night, expids[ii]),
        '--seed', str(seeds[ii])
    ])
    quickgen.main(quickgen_args)

DEBUG:obs.py:101:new_exposure: Generating 10 targets
DEBUG:targets.py:195:get_targets_parallel: Not Parallelizing get_targets for only 10 targets
DEBUG:targets.py:262:get_targets: Using random seed 1800636030
INFO:io.py:787:read_basis_templates: Reading /Users/ioannis/work/desi/spectro/templates/basis_templates/v2.3/star_templates_v2.1.fits
INFO:io.py:787:read_basis_templates: Reading /Users/ioannis/work/desi/spectro/templates/basis_templates/v2.3/bgs_templates_v2.1.fits
DEBUG:templates.py:361:_blurmatrix: Populating blur matrix with 1 unique velocity dispersion values.
DEBUG:simexp.py:371:simulate_spectra: loading specsim desi config
DEBUG:simexp.py:375:simulate_spectra: creating specsim desi simulator
DEBUG:specsim.py:39:get_simulator: Creating new  Simulator
INFO:simexp.py:396:simulate_spectra: MJD not in obsconditions, using DATE-OBS 2009-06-18T12:00:00.000
DEBUG:simexp.py:400:simulate_spectra: obsconditions MOONFRAC = 0.25
DEBUG:simexp.py:400:simulate_spectra: obsconditions MOONSE

/usr/local/anaconda3/envs/desi/lib/python3.5/site-packages/astropy/table/column.py:1096: MaskedArrayFutureWarning: setting an item on a masked array which has a shared mask will not copy the mask and also change the original mask array in the future.
Check the NumPy 1.11 release notes for more information.
  ma.MaskedArray.__setitem__(self, index, value)


INFO:io.py:147:write_simspec: DATE-OBS 2017-01-01T22:00:00 UTC
INFO:io.py:223:write_simspec: Writing /Users/ioannis/work/desi/spectro/sim/snesim-01/20170101/simspec-00001000.fits


INFO:obs.py:211:new_exposure: Wrote /Users/ioannis/work/desi/spectro/sim/snesim-01/20170101/fibermap-00001000.fits
Commenting out update_obslog!!!
INFO:quickgen.py:239:main: Reading fibermap file /Users/ioannis/work/desi/spectro/sim/snesim-01/20170101/fibermap-00001000.fits
DEBUG:simexp.py:518:get_source_types: bgs 7 targets
DEBUG:simexp.py:518:get_source_types: sky 2 targets
DEBUG:simexp.py:518:get_source_types: star 1 targets
INFO:quickgen.py:272:main: Initializing SpecSim with config desi
DEBUG:specsim.py:39:get_simulator: Creating new desi Simulator
INFO:quickgen.py:278:main: Reading input file /Users/ioannis/work/desi/spectro/sim/snesim-01/20170101/simspec-00001000.fits
INFO:quickgen.py:314:main: Only 10 spectra in input file
INFO:quickgen.py:677:main: Writing files for channel:b, spectrograph:0, spectra:0 to 10
INFO:quickgen.py:704:main: Wrote file /Users/ioannis/work/desi/spectro/redux/snesim-01/exposures/20170101/00001000/frame-b0-00001000.fits
INFO:quickgen.py:722:main: Wrote 

INFO:quickgen.py:722:main: Wrote file /Users/ioannis/work/desi/spectro/redux/snesim-01/exposures/20170101/00001000/cframe-r0-00001000.fits
INFO:quickgen.py:737:main: Wrote file /Users/ioannis/work/desi/spectro/redux/snesim-01/exposures/20170101/00001000/sky-r0-00001000.fits
INFO:quickgen.py:758:main: Wrote file /Users/ioannis/work/desi/spectro/redux/snesim-01/exposures/20170101/00001000/calib-r0-00001000.fits
INFO:quickgen.py:677:main: Writing files for channel:z, spectrograph:0, spectra:0 to 10
INFO:quickgen.py:704:main: Wrote file /Users/ioannis/work/desi/spectro/redux/snesim-01/exposures/20170101/00001000/frame-z0-00001000.fits
INFO:quickgen.py:722:main: Wrote file /Users/ioannis/work/desi/spectro/redux/snesim-01/exposures/20170101/00001000/cframe-z0-00001000.fits
INFO:quickgen.py:737:main: Wrote file /Users/ioannis/work/desi/spectro/redux/snesim-01/exposures/20170101/00001000/sky-z0-00001000.fits
INFO:quickgen.py:758:main: Wrote file /Users/ioannis/work/desi/spectro/redux/snesim-01

In [20]:
simspecfile = desisim.io.findfile('simspec', night, expids[0])

In [30]:
Table(fitsio.read(simspecfile, ext='OBSCONDITIONS'))

AIRMASS,EXPTIME,MOONALT,MOONFRAC,MOONSEP,SEEING
float64,float64,float64,float64,float64,float64
1.0,300.0,30.0,0.25,122.144765125,1.0


In [26]:
Table(fitsio.read(simspecfile, ext='TRUTH'))

OBJTYPE,SUBTYPE,TEMPLATEID,SEED,REDSHIFT,MAG,FLUX_G,FLUX_R,FLUX_Z,FLUX_W1,FLUX_W2,OIIFLUX,HBETAFLUX,EWOII,EWHBETA,D4000,VDISP,OIIDOUBLET,OIIIHBETA,OIIHBETA,NIIHBETA,SIIHBETA,ZMETAL,AGE,TEFF,LOGG,FEH
bytes10,bytes10,int32,int64,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32
BGS,,1486,615974893,0.126328,19.5,6.49462,15.8489,26.5979,20.2283,13.0409,-1.0,0.0,-1.0,0.0,1.68271,101.393,0.812913,-0.577526,0.0148598,0.0287722,-0.223423,-1.0,-1.0,-1.0,-1.0,-1.0
BGS,,453,2388722153,0.106159,19.5,6.81948,15.8489,26.0677,22.0677,15.2934,-1.0,0.0,-1.0,0.0,1.66012,101.393,0.844736,0.244524,0.441384,-0.471636,-0.451952,-1.0,-1.0,-1.0,-1.0,-1.0
BGS,,2106,1537356348,0.148127,19.5,9.75361,15.8489,19.2331,13.2235,9.96657,-1.0,5.36079e-16,-1.0,11.7057,1.15039,101.393,0.786791,0.806695,0.0114448,-1.56563,-1.0452,-1.0,-1.0,-1.0,-1.0,-1.0
SKY,,-1,-1,0.0,-1.0,0.0,0.0,0.0,0.0,0.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
FSTD,,533,615974893,0.000524233,16.3556,216.471,286.929,315.509,84.3881,47.2535,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,6000.0,4.0,-2.5
BGS,,3080,1395253339,0.125266,19.5,5.96754,15.8489,28.7677,27.9569,19.2741,-1.0,0.0,-1.0,0.0,1.91616,101.393,0.802047,-0.0335126,0.438712,-0.192215,-0.470078,-1.0,-1.0,-1.0,-1.0,-1.0
BGS,,3222,2109587489,0.285203,19.5,7.78251,15.8489,23.7197,25.5036,21.4877,-1.0,4.2466e-16,-1.0,9.71795,1.18718,101.393,0.827255,-0.320544,0.262373,0.0184941,0.167621,-1.0,-1.0,-1.0,-1.0,-1.0
BGS,,3144,1765536105,0.155676,19.5,5.40382,15.8489,29.7844,29.0348,20.1163,-1.0,0.0,-1.0,0.0,1.97526,101.393,0.681986,-0.272869,0.303555,0.112697,0.0176023,-1.0,-1.0,-1.0,-1.0,-1.0
SKY,,-1,-1,0.0,-1.0,0.0,0.0,0.0,0.0,0.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0


In [29]:
Table(fitsio.read(simspecfile, ext='FIBERMAP'))

OBJTYPE,TARGETCAT,BRICKNAME,TARGETID,DESI_TARGET,BGS_TARGET,MWS_TARGET,MAG [5],FILTER [5],SPECTROID,POSITIONER,LOCATION,DEVICE_LOC,PETAL_LOC,FIBER,LAMBDAREF,RA_TARGET,DEC_TARGET,RA_OBS,DEC_OBS,X_TARGET,Y_TARGET,X_FVCOBS,Y_FVCOBS,Y_FVCERR,X_FVCERR
bytes10,bytes20,bytes8,int64,int64,int64,int64,float32,bytes40,int32,int64,int32,int32,int32,int32,float32,float64,float64,float64,float64,float64,float64,float64,float64,float32,float32
BGS,,1300p187,7406197882867160669,1152921504606846976,2,0,20.4686 .. 19.7117,DECAM_G .. WISE_W2,0,95,95,95,0,0,5400.0,130.097807355,18.7478748417,130.097807355,18.7478748417,-4.13464340313,-176.016389199,-4.13464340313,-176.016389199,0.0,0.0
BGS,,1303p187,6257323788419429938,1152921504606846976,2,0,20.4156 .. 19.5387,DECAM_G .. WISE_W2,0,62,62,62,0,1,5400.0,130.276470593,18.6714981019,130.276470593,18.6714981019,-45.5863791815,-157.134697634,-45.5863791815,-157.134697634,0.0,0.0
BGS,,1303p187,8774121736903493301,1152921504606846976,2,0,20.0271 .. 20.0036,DECAM_G .. WISE_W2,0,102,102,102,0,2,5400.0,130.287445012,18.8215394291,130.287445012,18.8215394291,-48.26471976,-194.58679079,-48.26471976,-194.58679079,0.0,0.0
SKY,,1303p187,2033231820972197058,4294967296,0,0,0.0 .. 0.0,..,0,82,82,82,0,3,5400.0,130.216005387,18.7340760901,130.216005387,18.7340760901,-31.5795923296,-172.635369698,-31.5795923296,-172.635369698,0.0,0.0
STD,,1300p187,4224267853333412402,8589934592,0,0,16.6615 .. 18.3139,DECAM_G .. WISE_W2,0,131,131,131,0,4,5400.0,130.051321755,18.8492200271,130.051321755,18.8492200271,6.67200636809,-201.385810488,6.67200636809,-201.385810488,0.0,0.0
BGS,,1300p187,8093418779481343928,1152921504606846976,2,0,20.5605 .. 19.2876,DECAM_G .. WISE_W2,0,108,108,108,0,5,5400.0,130.023453433,18.7701510533,130.023453433,18.7701510533,13.1352857547,-181.585171402,13.1352857547,-181.585171402,0.0,0.0
BGS,,1300p187,8370050490150094129,1152921504606846976,2,0,20.2722 .. 19.1695,DECAM_G .. WISE_W2,0,119,119,119,0,6,5400.0,130.037330449,18.8097688704,130.037330449,18.8097688704,9.91914629057,-191.490598774,9.91914629057,-191.490598774,0.0,0.0
BGS,,1300p187,4845693505715794167,1152921504606846976,2,0,20.6682 .. 19.2411,DECAM_G .. WISE_W2,0,64,64,64,0,7,5400.0,130.188403599,18.6544586893,130.188403599,18.6544586893,-25.1362017254,-152.830188325,-25.1362017254,-152.830188325,0.0,0.0
SKY,,1303p187,7961454327049367540,4294967296,0,0,0.0 .. 0.0,..,0,72,72,72,0,8,5400.0,130.246304175,18.70280919,130.246304175,18.70280919,-38.5997312986,-164.880825023,-38.5997312986,-164.880825023,0.0,0.0


In [18]:
fitsio.FITS(desisim.io.findfile('simspec', night, expids[0]))



  file: /Users/ioannis/work/desi/spectro/sim/snesim-01/20170101/simspec-00001000.fits
  mode: READONLY
  extnum hdutype         hduname[v]
  0      IMAGE_HDU       WAVE
  1      IMAGE_HDU       FLUX
  2      IMAGE_HDU       SKYFLUX
  3      IMAGE_HDU       WAVE_B
  4      IMAGE_HDU       PHOT_B
  5      IMAGE_HDU       SKYPHOT_B
  6      IMAGE_HDU       WAVE_R
  7      IMAGE_HDU       PHOT_R
  8      IMAGE_HDU       SKYPHOT_R
  9      IMAGE_HDU       WAVE_Z
  10     IMAGE_HDU       PHOT_Z
  11     IMAGE_HDU       SKYPHOT_Z
  12     BINARY_TBL      TRUTH
  13     BINARY_TBL      FIBERMAP
  14     BINARY_TBL      OBSCONDITIONS